Welcome, to Punisher. After running your strategy, view the live results with Dash

```
$ python -m punisher.charts.dash_charts.dash_record --name default_backtest
```

## Define Strategy

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from punisher.common import *

from punisher.strategies.strategy import Strategy

In [2]:
class SimpleStrategy(Strategy):
    def __init__(self, asset, quantity):
        super().__init__()
        self.asset = asset
        self.quantity = quantity

    def log_all(self, orders, data, ctx, utc):
        self.logger = ctx.logger
        if self.logger is not None:
            self.log_epoch_time(utc)
            self.log_ohlcv(data, self.asset.symbol, ctx.exchange.id)
            self.log_orders(orders)
            self.log_performance(ctx)
            self.log_balance(ctx)
            self.log_positions(ctx)
            self.log_metrics(ctx)

    def handle_data(self, data, ctx):
        new_orders = []
        quantity = .05
        price = data.get('close', self.asset.symbol, ctx.exchange.id)
        current_time = data.get('utc')
        if random.random() > 0.5:
            order = order_manager.build_limit_buy_order(
                ctx.record.portfolio.balance, ctx.exchange, self.asset, quantity, price, current_time)
            new_orders.append(order)
        elif ctx.record.balance.get(self.asset.base)[BalanceType.FREE] > quantity:
            order = order_manager.build_market_sell_order(
                ctx.record.portfolio.balance, ctx.exchange, self.asset, quantity, current_time)
            new_orders.append(order)

        # Optionally cancel pending orders (LIVE trading)
        #pending_orders = ctx.exchange.fetch_open_orders(asset)
        cancel_ids = []

        # Add Metrics and OHLCV to Record
        self.update_metric('SMA', 5.0, ctx)
        self.update_metric('RSI', 10.0, ctx)
        self.update_ohlcv(data, ctx)
        self.log_all(new_orders, data, ctx, current_time)
        return {
            'new_orders': new_orders,
            'cancel_ids': cancel_ids
        }

## Backtest

### Backtest From Local CSV Files

In [ ]:
# Download some data from Binance
exchange_id = ex_cfg.BINANCE
timeframe = Timeframe.THIRTY_MIN
start = datetime.datetime(year=2018, month=1, day=1, hour=0)
end = datetime.datetime(year=2018, month=1, day=5, hour=0)
assets = [Asset(coins.ETH, coins.BTC), Asset(coins.BTC, coins.USDT)]
data_exchange = load_exchange(exchange_id)
ohlcv_feed.download_ohlcv([data_exchange], assets, timeframe, start, end)

In [ ]:
# Setup the Feed Based Paper Exchange
cash = 1.0
cash_currency = coins.BTC
balance = Balance(
    cash_currency=cash_currency,
    starting_cash=cash
)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=timeframe
)
portfolio = Portfolio(
    cash_currency=cash_currency,
    starting_balance=deepcopy(balance),
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
feed = OHLCVFileFeed(
    exchange_ids=[exchange_id],
    assets=assets,
    timeframe=timeframe,
    start=None, # Usually None for backtest, but its possible to filter the csv
    end=None
)
exchange = load_feed_based_paper_exchange(deepcopy(balance), feed, exchange_id)
strategy = SimpleStrategy(assets[0], quantity=.05)
experiment_name = 'default_backtest'

In [ ]:
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

In [ ]:
record

### Backtest from External Exchange Data

In [ ]:
cash = 1.0
cash_currency = coins.BTC
balance = Balance(
    cash_currency=cash_currency,
    starting_cash=cash
)
exchange = load_ccxt_based_paper_exchange(balance, ex_cfg.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    cash_currency=cash_currency,
    starting_balance=balance,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)

assets = [Asset(coins.ETH, coins.BTC), Asset(coins.LTC, coins.BTC)]
feed = OHLCVExchangeFeed(
    exchanges=[exchange],
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow()-datetime.timedelta(days=1),
    end=None
)
strategy = SimpleStrategy(assets[0], quantity=.05)

In [ ]:
experiment_name = 'default_backtest_exchange'
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

## Simulate Orders

In [3]:
cash = 1.0
cash_currency = coins.BTC
balance = Balance(
    cash_currency=coins.BTC,
    starting_cash=cash
)
exchange = load_ccxt_based_paper_exchange(balance, ex_cfg.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    cash_currency=cash_currency,
    starting_balance=deepcopy(balance),
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)

assets = [Asset(coins.ETH, coins.BTC), Asset(coins.LTC, coins.BTC)]
feed = OHLCVExchangeFeed(
    exchanges=[exchange],
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow(),
    end=None
)
strategy = SimpleStrategy(assets[0], quantity=.05)

START 2018-01-31 17:43:24.896092
END 2018-01-31 17:43:24.896110
Downloading: ETH/BTC
Downloaded rows: 0
Downloading: LTC/BTC
Downloaded rows: 0
Downloading: BTC/USDT
Downloaded rows: 0


In [4]:
experiment_name = 'default_simulate'
runner.simulate(experiment_name, exchange, balance, portfolio, feed, strategy)

Downloading: ETH/BTC
Downloaded rows: 0
Downloading: LTC/BTC
Downloaded rows: 0
Downloading: BTC/USDT
Downloaded rows: 0
No data after prior poll: 2018-01-31 17:43:24.896092
Downloading: ETH/BTC
Downloaded rows: 0
Downloading: LTC/BTC
Downloaded rows: 0
Downloading: BTC/USDT
Downloaded rows: 0
No data after prior poll: 2018-01-31 17:43:24.896092
Downloading: ETH/BTC
Downloaded rows: 0
Downloading: LTC/BTC
Downloaded rows: 0
Downloading: BTC/USDT
Downloaded rows: 0
No data after prior poll: 2018-01-31 17:43:24.896092
Downloading: ETH/BTC
Downloaded rows: 1
Downloading: LTC/BTC


---------------------------------------
Epoch 1 - Timestep: 2018-01-31T17:44:00
---------------------------------------
OHLCV
    O: 0.1104 | C: 0.1104 | V: 26.2 | T: 2018-01-31T17:44:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.1104 | Quantity: 0.0500
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 1.0}
    ETH - {'free': 0.0, 'used': 0.0, 'total': 0.0}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00


Downloaded rows: 1
Downloading: BTC/USDT
Downloaded rows: 1
TIME 2018-01-31 17:44:14.692107
Inspecting trade times...
trade time 2018-01-31 17:44:14.692557
last_update_time 2018-01-31 17:43:59.216479
Downloading: ETH/BTC
Downloaded rows: 1
Downloading: LTC/BTC
Downloaded rows: 1
Downloading: BTC/USDT
Downloaded rows: 1
No data after prior poll: 2018-01-31 17:44:00
Downloading: ETH/BTC
Downloaded rows: 1
Downloading: LTC/BTC
Downloaded rows: 1
Downloading: BTC/USDT
Downloaded rows: 1
No data after prior poll: 2018-01-31 17:44:00
Downloading: ETH/BTC
Downloaded rows: 1
Downloading: LTC/BTC
Downloaded rows: 2
Downloading: BTC/USDT
Downloaded rows: 2
No data after prior poll: 2018-01-31 17:44:00
Downloading: ETH/BTC
Downloaded rows: 2
Downloading: LTC/BTC


---------------------------------------
Epoch 2 - Timestep: 2018-01-31T17:45:00
---------------------------------------
OHLCV
    O: 0.1104 | C: 0.1104 | V: 17.6 | T: 2018-01-31T17:45:00
ORDERS
PERFORMANCE
    Cash: 0.9945 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 0.99448199999999998, 'used': 0.0, 'total': 0.99448199999999998}
    ETH - {'free': 0.05, 'used': 0.0, 'total': 0.05}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.05, 'cost_price': 0.11035999999999999, 'latest_price': 0.11035999999999999, 'fee': 0.0}
METRICS
    SMA: 5.00
    RSI: 10.00


Downloaded rows: 2
Downloading: BTC/USDT
Downloaded rows: 2
Downloading: ETH/BTC
Downloaded rows: 1
Downloading: LTC/BTC
Downloaded rows: 1
Downloading: BTC/USDT
Downloaded rows: 1
No data after prior poll: 2018-01-31 17:45:00
Downloading: ETH/BTC
Downloaded rows: 1
Downloading: LTC/BTC
Downloaded rows: 1
Downloading: BTC/USDT
Downloaded rows: 1
No data after prior poll: 2018-01-31 17:45:00
Downloading: ETH/BTC
Downloaded rows: 2
Downloading: LTC/BTC


...

---------------------------------------
Epoch 42 - Timestep: 2018-01-31T18:25:00
---------------------------------------
OHLCV
    O: 0.1105 | C: 0.1104 | V: 5.3 | T: 2018-01-31T18:25:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.1104 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9779 Total Val: 1.0000 PnL: -0.0000 Returns: -0.0000
BALANCE
    BTC - {'free': 0.97790795000000019, 'used': 0.0, 'total': 0.97790795000000019}
    ETH - {'free': 0.2, 'used': 0.0, 'total': 0.2}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.2, 'cost_price': 0.11042045370615781, 'latest_price': 0.110377, 'fee': 0.0}
METRICS
    SMA: 5.00
    RSI: 10.00


Downloaded rows: 2
Downloading: BTC/USDT
Downloaded rows: 2
TIME 2018-01-31 18:25:03.660133
Inspecting trade times...
trade time 2018-01-31 18:25:03.660381
last_update_time 2018-01-31 18:24:48.156742
Downloading: ETH/BTC
Downloaded rows: 1
Downloading: LTC/BTC
Downloaded rows: 1
Downloading: BTC/USDT
Downloaded rows: 1
No data after prior poll: 2018-01-31 18:25:00
Downloading: ETH/BTC
Downloaded rows: 1
Downloading: LTC/BTC
Downloaded rows: 1
Downloading: BTC/USDT
Downloaded rows: 1
No data after prior poll: 2018-01-31 18:25:00
Downloading: ETH/BTC
Downloaded rows: 1
Downloading: LTC/BTC
Downloaded rows: 1
Downloading: BTC/USDT
Downloaded rows: 1
No data after prior poll: 2018-01-31 18:25:00
Downloading: ETH/BTC
Downloaded rows: 2
Downloading: LTC/BTC


---------------------------------------
Epoch 43 - Timestep: 2018-01-31T18:26:00
---------------------------------------
OHLCV
    O: 0.1105 | C: 0.1105 | V: 2.8 | T: 2018-01-31T18:26:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9724 Total Val: 1.0000 PnL: -0.0000 Returns: -0.0000
BALANCE
    BTC - {'free': 0.97238880000000016, 'used': 0.0, 'total': 0.97238880000000016}
    ETH - {'free': 0.25, 'used': 0.0, 'total': 0.25}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.25, 'cost_price': 0.11041296296492625, 'latest_price': 0.110383, 'fee': 0.0}
METRICS
    SMA: 5.00
    RSI: 10.00


Downloaded rows: 1
Downloading: BTC/USDT
Downloaded rows: 2
TIME 2018-01-31 18:26:06.066622
Inspecting trade times...
trade time 2018-01-31 18:26:06.067285
last_update_time 2018-01-31 18:25:50.433389
Downloading: ETH/BTC
Downloaded rows: 1
Downloading: LTC/BTC
Downloaded rows: 1
Downloading: BTC/USDT
Downloaded rows: 1
No data after prior poll: 2018-01-31 18:26:00
Downloading: ETH/BTC


RequestTimeout: binance GET https://api.binance.com/api/v1/klines?symbol=ETHBTC&interval=1m&limit=500&startTime=1517423160000 request timeout

## Live Trade

In [ ]:
exchange = load_exchange(ex_cfg.BINANCE)
cash = exchange.fetch_balance().get(coins.BTC)[BalanceType.FREE]
cash_currency = coins.BTC
print(coins.BTC, "cash", cash)

In [ ]:
balance = Balance(
    cash_currency=coins.BTC,
    starting_cash=cash
)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    cash_currency=cash_currency,
    starting_balance=balance,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
assets = [Asset(coins.ETH, coins.BTC), Asset(coins.LTC, coins.BTC)]
feed = OHLCVExchangeFeed(
    exchanges=[exchange],
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=None,
    end=None
)
experiment_name = 'default_live'

In [ ]:
runner.live(experiment_name, exchange, balance, portfolio, feed, strategy)